# Building a Spam Filter with Naive Bayes
Building a spam filter for SMS messages is a practical and useful application of machine learning algorithms that can help users avoid being bombarded with unwanted messages and improve their overall experience.

![](https://www.informaticsinc.com/application/files/4515/2718/5763/iStock-122143117.jpg)

In this project, we will delve into the practical application of algorithms by building a spam filter for SMS messages. The aim of this project is to train a computer to classify SMS messages as either spam or non-spam with an accuracy greater than 80%. This will be done by:
- leveraging human knowledge of how messages are classified
- using the information above to estimate the probability of a new message being either spam or non-spam. 
- having the computer classify messages as either spam or non-spam based on the probability values.

To achieve this, we will use the **Multinomial Naive Bayes** algorithm along with a dataset of 5,572 SMS messages that have already been classified by humans. Tiago A. Almeida and José María Gómez Hidalgo compiled the [dataset](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection), which is available for download from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

> _If you want to learn more about how the data used in this project was collected, you can visit [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition). Additionally, the page also contains papers authored by the creators of the dataset, which you may find useful._

> _Due to the nature of spam messages, the dataset contains content that may be offensive to some users._

The dataset comprises pre-labelled SMS messages that have been labelled as either spam or non-spam and will be used to train the algorithm. Once the algorithm is trained, it can be used to classify new messages as spam or non-spam.

## Importing Libraries
---

In [1]:
import requests
import zipfile
import io
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set the default behaviour of plots
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Exploring the Dataset
---

### 1. Downloading the dataset
First, we will download the file from the url using the [requests](https://pypi.org/project/requests/) library, then extract the contents into a local folder. To extract these contents, we use the [zipfile](https://docs.python.org/3/library/zipfile.html) library.

In [2]:
# Pull the data from the url
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
response = requests.get(url)

# Extract the contents into a folder locally
folder_name = url.split('/')[-1][:-4]
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall("./" + folder_name)

# Check if the extraction process was successful
! ls

notebook.ipynb    smsspamcollection


### 2. Checking the file structure
Next, we will check how the file contents are structured (by displaying the first five rows of the file). Understanding how the data is structured makes it easier to work with the data in Pandas.

In [3]:
# Preview from the command line
! cd smsspamcollection; head -n 5 SMSSpamCollection; cd ../

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though


The data lacks a header row, and the columns are separated by tabs. We can account for these structural attributes when reading the data with Pandas.
### 3. Exploring with Pandas
Finally, we can read the dataset and explore its attributes further:

In [4]:
# Read-in the data with pandas
df = pd.read_csv("./smsspamcollection/SMSSpamCollection", 
                 sep= "\t", 
                 header= None, 
                 names= ["label", "SMS"]
                )
                

# Explore dataset attributes
print(f'STRUCTURE:\nThe SMS dataset has {df.shape[0]} rows and {df.shape[1]} columns.\n')
print(f'COMPLETENESS:\nThe dataset has {df.isnull().sum().sum()} number of null values.', '\n')

print('PREVIEW:\nPrinting the first five rows of the dataset...')
df.head()

STRUCTURE:
The SMS dataset has 5572 rows and 2 columns.

COMPLETENESS:
The dataset has 0 number of null values. 

PREVIEW:
Printing the first five rows of the dataset...


,label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Check the proportion of spam vs ham messages
print('SPAM VS HAM (NON-SPAM):\n\
Printing the proportion on spam vs ham messages...')
df.label.value_counts(normalize= True).to_frame(name='proportion')

SPAM VS HAM (NON-SPAM):
Printing the proportion on spam vs ham messages...


,proportion
ham,0.865937
spam,0.134063


Of the total number of SMS messages in the dataset, about 87% are classified as "ham" and the remaining 13% are classified as "spam". This indicates that the dataset is representative of the actual message distribution in practice, where most of the messages people receive are legitimate messages ("ham") rather than unsolicited messages ("spam"). 

Therefore, the dataset can be considered appropriate for training our filter to accurately distinguish between spam and ham messages, which will ultimately result in an effective spam filter for SMS messages.

## Creating the Training and Test Datasets
---
We found that about 87% of the messages in the dataset are ham and 13% are spam. To build a spam filter, it is helpful to first design a test to ensure that the filter works effectively. 

We will split our dataset into two categories as follows:
1. A training set for teaching the computer how to classify messages.
2. A test set for evaluating the performance of the spam filter. 
> _We'll keep **80%** for training and **20%** for testing._ 

Our aim is to create a spam filter that can classify new messages with an accuracy greater than 80%. To test this, we will compare the results of the algorithm's classification with that of the human classification on the test set.

In [6]:
# Randomize entire dataframe
randomized_df = df.sample(frac=1, random_state=1)

# Create training and test set
num_records = randomized_df.shape[0]
train_set_size = round(num_records * 0.8)

train_set = randomized_df.iloc[:train_set_size].reset_index(drop=True)
test_set = randomized_df.iloc[train_set_size:].reset_index(drop=True)

# Verify training set as 80% and test set as 20% of dataframe
assert train_set.shape[0] == round(0.8 * randomized_df.shape[0])
assert test_set.shape[0] == round(0.2 * randomized_df.shape[0])
print(f'The training set has {train_set.shape[0]} records.')
print(f'The test set has {test_set.shape[0]} records.')

The training set has 4458 records.
The test set has 1114 records.


In the next step, we will examine the proportion of spam and ham messages in both the training and test sets. We anticipate that these proportions will be similar to the ones found in the complete dataset, where approximately 87% of the messages are ham and the remaining 13% are spam.

In [7]:
print("Calculating spam vs ham proportions...")

pd.concat([df.label.value_counts(normalize=True).round(2),
                     train_set.label.value_counts(normalize=True).round(2),
                     test_set.label.value_counts(normalize=True).round(2)], 
                     axis=1, 
                     keys=['full_dataset', 'train_set', 'test_set']
         )

Calculating spam vs ham proportions...


,full_dataset,train_set,test_set
ham,0.87,0.87,0.87
spam,0.13,0.13,0.13


Since the proportions resemble those of the full dataset, our sampling method is deemed representative, and we can proceed.

## Cleaning the Dataset
---
The Naive Bayes algorithm we'll be using for the spam filter is based on the following two equations, where the letter _w_ denotes words in each SMS:

$$ P(Spam\mid w_1, w_2,...,w_n) \propto P(Spam) \cdot \prod\limits_{i=1}^{n}P(w_i\mid Spam)$$

$$ P(Ham\mid w_1, w_2,...,w_n) \propto P(Ham) \cdot \prod\limits_{i=1}^{n}P(w_i\mid Ham)$$


By analyzing these equations, we can see that each word in the message plays an essential role in helping the algorithm to classify messages as spam or ham. As a result, we will treat every word in each SMS as a variable.

However, we have to consider the fact that our dataset should be tidy for our purposes. In general, for data to be considered tidy, it must meet three criteria:

1. Each column should be a variable. In our case, we will treat every unique word in each SMS as a variable, so we have multiple columns for each SMS.
2. Every row should be an observation. In our case, each row will describe an SMS, with the number of times each unique word occurs in it.
3. Third, every cell should contain a single value.

> _To get a thorough knowledge of data tidiness rules, read [this article](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html)._


### 1. Letter Case and Punctuation
In addition to treating every word as a variable in our Naive Bayes algorithm, we will also make the algorithm case and punctuation insensitive. This means that the algorithm will not differentiate between uppercase and lowercase letters, and will ignore any punctuation marks in the messages.

For example, the words "FREE" and "free" will be treated as the same word, and the algorithm will not distinguish between them. Similarly, the presence of a punctuation mark such as a period or exclamation mark at the end of a sentence will not affect the classification of the message as spam or ham.

By making the algorithm case and punctuation insensitive, we can simplify the classification process and reduce the number of variables that the algorithm needs to consider. This can help improve the accuracy of the classification by reducing the potential for errors caused by case or punctuation differences in similar words or phrases.

In [8]:
train_set["SMS"] = (train_set.SMS
                         .str.replace("\W", " ", regex=True)
                         .str.lower()
                     )
                     
train_set.head(5)

,label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### 2. Creating the Vocabulary
> _The vocabulary represents the list of all of the unique words that occur in all SMS messages of our training set._

In [9]:
# Split each SMS into a python list
train_set["SMS"] = train_set["SMS"].str.split()

# Append all the words in each list into a vocabulary
vocabulary = []
train_set["SMS"].apply(lambda x: [vocabulary.append(item) for item in x]);

# Only retain the unique words in the vocabulary
vocabulary = list(set(vocabulary))

# Check the number of unique words in the vocabulary
print("Our vocabulary has {:,} unique words".format(len(vocabulary)))

Our vocabulary has 7,783 unique words


### 3. The Final Training Set
We can create a tidy dataframe by using the words in our vocabulary. This dataframe will have distinct columns for each unique word. Each column will store the number of times the word appears in an SMS message.

In [10]:
# Initialize a dictionary with each vocabulary word as keys and the length of the dataset as values
word_count_per_SMS = {word: [0]* train_set.shape[0] for word in vocabulary}

# Estimate the number of times each unique word apears in each SMS message
for index, words in enumerate(train_set.SMS):
    for word in (words):
        word_count_per_SMS[word][index] += 1

# Covert results into a Pandas dataframe
word_count_df = pd.DataFrame(word_count_per_SMS)
word_count_df.head()

,wocay,sip,spelling,subscribed,door,nursery,transcribing,itcould,3ss,csh11,...,reserve,financial,senthil,sense,capacity,werethe,sensible,starshine,voda,1172
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Concatenate the training set datframe with the word counts dataframe
train_set_clean = pd.concat([train_set, word_count_df], axis=1)
train_set_clean.head()

,label,SMS,wocay,sip,spelling,subscribed,door,nursery,transcribing,itcould,...,reserve,financial,senthil,sense,capacity,werethe,sensible,starshine,voda,1172
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First
---
After completing the data cleaning process and obtaining a training dataset, we can now start building the spam filter using the Naive Bayes algorithm. In order to classify new messages, the algorithm requires the probability values of the two equations provided below:

$$ P(Spam\mid w_1, w_2,...,w_n) \propto P(Spam) \cdot \prod\limits_{i=1}^{n}P(w_i\mid Spam)$$

$$ P(Ham\mid w_1, w_2,...,w_n) \propto P(Ham) \cdot \prod\limits_{i=1}^{n}P(w_i\mid Ham)$$

Moreover, in order to calculate _P(w<sub>i</sub>|Spam)_ and _P(w<sub>i</sub>|Ham)_ in the equations above, we need to utilize the following formulas:

$$ P(w_i\mid Spam) = \frac{N_{(w_i\mid Spam)} + \alpha}{N_{spam} + \alpha \cdot N_{Vocabulary}} $$

$$ P(w_i\mid Ham) = \frac{N_{(w_i\mid Ham)} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}} $$

It's important to note that some of the terms in these four equations will have identical values for every new message. Therefore, we can calculate the values of these terms once and avoid re-computing them every time a new message is received. To achieve this, we will use our training dataset to calculate:

1. The probability of spam and ham messages in the training set: P(Spam) and P(Ham).

2. The total number of words in all spam messages and ham messages: N<sub><i>Spam</i></sub> and N<sub><i>Ham</i></sub>.

3. The total number of unique words in all spam messages and ham messages: N<sub><i>Vocabulary</i></sub>.

> _For the smoothing constant ($\alpha$), we will use Laplace smoothing and set $\alpha$ = 1._

Once we have these calculations, we can easily determine the probability of a new message being spam or ham based on its word frequency. This approach will allow the Naive Bayes algorithm to more efficiently and accurately classify new messages.


In [12]:
# Create a dictionary to hold the constants
constants = {
    # Probability of spam
    'p_spam': train_set_clean.label.value_counts(normalize=True)['spam'],
    # Probability of non spam
    'p_ham':  train_set_clean.label.value_counts(normalize=True)['ham'],
    # number of words in all spam messages
    'n_spam': train_set_clean.groupby('label').sum().sum(axis=1)['spam'],
    # number of words in all non spam messages
    'n_ham': train_set_clean.groupby('label').sum().sum(axis=1)['ham'],
    # number of unique words in vocabulary
    'n_vocabulary': len(word_count_per_SMS),
    # Laplace smooting constant
    'alpha': 1
}

# Preview the values of the constants
print('Creating a table of constants...')
pd.DataFrame(constants, index=['values'])

Creating a table of constants...


,p_spam,p_ham,n_spam,n_ham,n_vocabulary,alpha
values,0.13459,0.86541,15190,57237,7783,1


## Calculating Parameters
---
Using the constant terms calculated earlier, we can proceed to calculate the parameters for _P(w<sub>i</sub>|Spam)_ and _P(w<sub>i</sub>|Ham)_, which will be the conditional probability values for each word in our vocabulary. To calculate these parameters, we will use the following formulas:

$$ P(w_i\mid Spam) = \frac{N_{(w_i\mid Spam)} + \alpha}{N_{spam} + \alpha \cdot N_{Vocabulary}} $$

$$ P(w_i\mid Ham) = \frac{N_{(w_i\mid Ham)} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}} $$

In [13]:
# Initialize a dictionary of spam and ham parameters
spam_parameters = {item: 0 for item in vocabulary}
ham_parameters = {item: 0 for item in vocabulary}

# Isolate spam and ham messages from the training set
spam_messages = train_set_clean.loc[train_set_clean['label'] == 'spam']
ham_messages = train_set_clean.loc[train_set_clean['label'] == 'ham']

# Calculate parameters
for word in vocabulary:
    p_word_given_spam = ((spam_messages[word].sum() + constants['alpha'])/
                         (constants['n_spam'] + constants['alpha']*constants['n_vocabulary'])
                        )
    
    p_word_given_ham  = ((ham_messages[word].sum() + constants['alpha'])/
                         (constants['n_ham'] + constants['alpha']*constants['n_vocabulary'])
                        )
    
    spam_parameters[word] = p_word_given_spam
    ham_parameters[word] = p_word_given_ham

## Common Words in Spam Messages
---